## Langchain Expression Language Basics

-  LangChain Expression Language is that any two runnables can be "chained" together into sequences. 
- The output of the previous runnable's .invoke() call is passed as input to the next runnable.
- This can be done using the pipe operator (|), or the more explicit .pipe() method, which does the same thing.

- Type of LCEL Chains
    - SequentialChain
    - Parallel Chain
    - Router Chain
    - Chain Runnables
    - Custom Chain (Runnable Sequence)

In [35]:
from dotenv import load_dotenv

load_dotenv('./../env')

True

### Sequential LCEL Chain

In [36]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
                                        SystemMessagePromptTemplate,
                                        HumanMessagePromptTemplate,
                                        ChatPromptTemplate
                                        )

base_url = "http://localhost:11434"
model = 'llama3.2:1b'

llm = ChatOllama(base_url=base_url, model=model)
llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [37]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)

question = template.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})

response = llm.invoke(question)
print(response.content)    

Here's a brief overview of the solar system:

1. Sun is at the center, providing light and heat.
2. Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune are planets that orbit around the sun.
3. The four inner planets (Mercury, Venus, Earth, Mars) are rocky, while the outer planets (Jupiter, Saturn, Uranus, Neptune) are gas giants.
4. Pluto is now classified as a dwarf planet because it's much smaller than other planets.
5. Asteroids and comets are small objects that orbit the sun in our solar system.


In [38]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)

chain = template | llm


In [39]:
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's an overview of the solar system in 5 key points:

1. **Distance from Earth**: Our solar system is about 93 million miles (150 million kilometers) away from our star, the Sun.

2. **Planets' Size and Composition**: Planets vary greatly in size and composition, ranging from small Mercury to large Jupiter.

3. **Moons of Planets**: Many planets have moons that orbit around them, providing unique features and environments for scientific study.

4. **Orbit Patterns**: Planets follow elliptical orbits, which means their distance from the Sun varies throughout the year, affecting the amount of sunlight they receive.

5. **Age of Solar System**: The solar system is estimated to be about 4.6 billion years old, with most planets forming from debris left over after a massive collision in the early days of the solar system.


In [40]:
response = chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 5})
print(response.content)

Here's an overview of the solar system in 5 key points:

1. The Sun is at the center of our solar system, making up about 99.8% of its mass.
2. The four planets closest to the Sun are Mercury, Venus, Earth, and Mars, with their orbits gradually increasing in distance.
3. Jupiter is the largest planet, with a massive atmosphere and dozens of moons orbiting it.
4. Saturn's prominent ring system is made up of ice and rock particles, while Uranus and Neptune have strong winds and icy surfaces.
5. Pluto is now classified as a dwarf planet, but was previously considered part of the inner solar system due to its proximity to the Sun and other planets.


In [41]:
response

AIMessage(content="Here's an overview of the solar system in 5 key points:\n\n1. The Sun is at the center of our solar system, making up about 99.8% of its mass.\n2. The four planets closest to the Sun are Mercury, Venus, Earth, and Mars, with their orbits gradually increasing in distance.\n3. Jupiter is the largest planet, with a massive atmosphere and dozens of moons orbiting it.\n4. Saturn's prominent ring system is made up of ice and rock particles, while Uranus and Neptune have strong winds and icy surfaces.\n5. Pluto is now classified as a dwarf planet, but was previously considered part of the inner solar system due to its proximity to the Sun and other planets.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-01T12:54:42.94655833Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14121304009, 'load_duration': 36794733, 'prompt_eval_count': 47, 'prompt_eval_duration': 888000000, 'eval_count': 147, 'eval_duration': 13194000000, 'mes

In [42]:
from langchain_core.output_parsers import StrOutputParser

In [43]:
chain = template | llm | StrOutputParser()
response = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 5})
print(response)

Here's a brief overview of the solar system:

• Our solar system is made up of eight planets (Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter).
• The sun is at the center of our solar system.
• Pluto was previously considered a planet but is now classified as a dwarf planet.
• Asteroids are small rocky objects that orbit the sun in space.
• Comets are icy bodies that release gas and dust when they approach the sun.


In [44]:
response

"Here's a brief overview of the solar system:\n\n• Our solar system is made up of eight planets (Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter).\n• The sun is at the center of our solar system.\n• Pluto was previously considered a planet but is now classified as a dwarf planet.\n• Asteroids are small rocky objects that orbit the sun in space.\n• Comets are icy bodies that release gas and dust when they approach the sun."

### Chaining Runnables (Chain Multiple Runnables)

- We can even combine this chain with more runnables to create another chain.
- Let's see how easy our generated output is?

In [45]:
chain

ChatPromptTemplate(input_variables=['points', 'school', 'topics'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['school'], input_types={}, partial_variables={}, template='You are {school} teacher. You answer in short sentences.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['points', 'topics'], input_types={}, partial_variables={}, template='tell me about the {topics} in {points} points'), additional_kwargs={})])
| ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
| StrOutputParser()

In [46]:
analysis_prompt = ChatPromptTemplate.from_template('''analyze the following text: {response}
                                                   You need tell me that how difficult it is to understand.
                                                   Answer in one sentence only.
                                                   ''')

fact_check_chain = analysis_prompt | llm | StrOutputParser()
output = fact_check_chain.invoke({'response': response})
print(output)

The text's clarity could be improved by providing a brief explanation or illustration of the concepts, making it easier for non-experts to grasp.


In [47]:
composed_chain = {"response": chain} | analysis_prompt | llm | StrOutputParser()

output = composed_chain.invoke({'school': 'phd', 'topics': 'solar system', 'points': 5})
print(output)

I would say that this text is moderately easy to understand due to its clear and concise language, as well as the specific examples and concepts presented.


### Parallel LCEL Chain
- Parallel chains are used to run multiple runnables in parallel.
- The final return value is a dict with the results of each value under its appropriate key.

In [48]:
system = SystemMessagePromptTemplate.from_template('You are {school} teacher. You answer in short sentences.')

question = HumanMessagePromptTemplate.from_template('tell me about the {topics} in {points} points')


messages = [system, question]
template = ChatPromptTemplate(messages)
fact_chain = template | llm | StrOutputParser()

output = fact_chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2})
print(output)

Here's an overview of the solar system:

* The Sun is at the center, with eight planets and many smaller bodies like moons and asteroids orbiting around it.
* Each planet has its own unique features, such as Mercury being closest to the Sun, Venus being hot, Earth being our home, Mars being rocky, Jupiter being largest, Saturn having rings, Uranus being icy, and Neptune being farthest from the Sun.


In [49]:
question = HumanMessagePromptTemplate.from_template('write a poem on {topics} in {sentences} lines')


messages = [system, question]
template = ChatPromptTemplate(messages)
poem_chain = template | llm | StrOutputParser()

output = poem_chain.invoke({'school': 'primary', 'topics': 'solar system', 'sentences': 2})
print(output)

The sun is at the center so bright,
Eight planets orbit around it with great might.


In [50]:
from langchain_core.runnables import RunnableParallel

In [51]:
chain = RunnableParallel(fact = fact_chain, poem = poem_chain)

In [52]:
output = chain.invoke({'school': 'primary', 'topics': 'solar system', 'points': 2, 'sentences': 2})
print(output['fact'])
print('\n\n')
print(output['poem'])

The solar system consists of eight planets, dwarf planets, asteroids, comets, and other objects.

* The eight planets in our solar system are: Mercury, Mars, Venus, Earth, Neptune, Uranus, Saturn, and Jupiter.
* The gas giants are four: Jupiter, Saturn, Uranus, and Neptune.



The sun at center, planets spin,
Eight orbits around, their paths to win.


### Chain Router
- The router chain is used to route the output of a previous runnable to the next runnable based on the output of the previous runnable.

In [53]:
prompt = """Given the user review below, classify it as either being about `Positive` or `Negative`.
            Do not respond with more than one word.

            Review: {review}
            Classification:"""

template = ChatPromptTemplate.from_template(prompt)

chain = template | llm | StrOutputParser()

review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."
# review = "I am not happy with the service. It is not good."
chain.invoke({'review': review})

'Negative'

In [54]:
positive_prompt = """
                You are expert in writing reply for positive reviews.
                You need to encourage the user to share their experience on social media.
                Review: {review}
                Answer:"""

positive_template = ChatPromptTemplate.from_template(positive_prompt)
positive_chain = positive_template | llm | StrOutputParser()

In [55]:
negative_prompt = """
                You are expert in writing reply for negative reviews.
                You need first to apologize for the inconvenience caused to the user.
                You need to encourage the user to share their concern on following Email:'udemy@kgptalkie.com'.
                Review: {review}
                Answer:"""


negative_template = ChatPromptTemplate.from_template(negative_prompt)
negative_chain = negative_template | llm | StrOutputParser()

In [56]:
def rout(info):
    if 'positive' in info['sentiment'].lower():
        return positive_chain
    else:
        return negative_chain

In [57]:
# rout({'sentiment': 'negetive'})

In [58]:
from langchain_core.runnables import RunnableLambda

In [59]:
full_chain = {"sentiment": chain, 'review': lambda x: x['review']} | RunnableLambda(rout)

In [60]:
full_chain

{
  sentiment: ChatPromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review'], input_types={}, partial_variables={}, template='Given the user review below, classify it as either being about `Positive` or `Negative`.\n            Do not respond with more than one word.\n\n            Review: {review}\n            Classification:'), additional_kwargs={})])
             | ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')
             | StrOutputParser(),
  review: RunnableLambda(lambda x: x['review'])
}
| RunnableLambda(rout)

In [61]:
# review = "Thank you so much for providing such a great plateform for learning. I am really happy with the service."
review = "I am not happy with the service. It is not good."

output = full_chain.invoke({'review': review})
print(output)

Here's a potential reply:

"Dear [User],

I'm so sorry to hear that your experience with us was not satisfactory. We understand how frustrating it can be when something isn't meeting our high standards, and we apologize for any inconvenience this has caused.

We take all feedback seriously and would like to assure you that we're committed to providing the best possible service. However, in this case, we fell short of expectations. Please know that we value your business and are taking immediate action to address the issues you've raised.

As a token of our apology, we'd like to invite you to share your concerns with us directly. Your feedback is invaluable in helping us improve and provide better services to all our students. You can reach out to us through the contact email provided below:

Udemy
kgptalkie.com

Please take a few minutes to share your thoughts on what we could do differently, and we'll review them carefully. Your input will play a significant role in shaping our future

### Make Custom Chain Runnables with RunnablePassthrough and RunnableLambda
- This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.



In [62]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [63]:
def char_counts(text):
    return len(text)

def word_counts(text):
    return len(text.split())

prompt = ChatPromptTemplate.from_template("Explain these inputs in 5 sentences: {input1} and {input2}")

In [64]:
prompt

ChatPromptTemplate(input_variables=['input1', 'input2'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input1', 'input2'], input_types={}, partial_variables={}, template='Explain these inputs in 5 sentences: {input1} and {input2}'), additional_kwargs={})])

In [65]:
chain = prompt | llm | StrOutputParser()

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})

print(output)

This statement appears to be a simple declaration of the relationship between our planet, Earth, and its primary source of light and energy, the Sun. The concept that Earth orbits around the Sun can be understood through scientific theories such as heliocentrism, which suggests that the planets in our solar system move around the Sun due to gravity. In this context, the Sun is not considered a "planet" itself but rather the star at the center of our solar system. The Sun's position and energy output are essential for sustaining life on Earth, making it an integral part of our ecosystem. This fundamental understanding of our planet's relationship with its star is a cornerstone of astronomy and space science.


In [66]:
chain = prompt | llm | StrOutputParser() | {'char_counts': RunnableLambda(char_counts), 
                                            'word_counts': RunnableLambda(word_counts), 
                                            'output': RunnablePassthrough()}

output = chain.invoke({'input1': 'Earth is planet', 'input2': 'Sun is star'})

print(output)

{'char_counts': 701, 'word_counts': 112, 'output': 'The inputs you provided are a basic understanding of our solar system. The Earth is indeed considered the third planet from the Sun, often referred to as the "terrestrial planet" due to its solid surface. In contrast, the Sun is a massive ball of hot, glowing gas, specifically designated as a star by astronomers due to its incredible energy output and life-giving properties for our planet. The relationship between these two celestial bodies is governed by the fundamental forces of physics, such as gravity and electromagnetism, which hold them together in our solar system. By understanding this basic connection, we can appreciate the intricate web of relationships that exist within our cosmos.'}


### Custom Chain using `@chain` decorator

In [67]:
from langchain_core.runnables import chain

In [68]:
@chain
def custom_chain(params):
    return {
        'fact': fact_chain.invoke(params),
        'poem': poem_chain.invoke(params),
    }


params = {'school': 'primary', 'topics': 'solar system', 'points': 2, 'sentences': 2}
output = custom_chain.invoke(params)
print(output['fact'])
print('\n\n')
print(output['poem'])

Here's what I know:

• The sun is at the center of our solar system, with planets orbiting around it in different paths.
• The four inner planets - Mercury, Venus, Earth, and Mars - are rocky and have a similar size to the sun, while the outer planets - Jupiter, Saturn, Uranus, and Neptune - are gas giants.



The sun at center, planets spin,
A celestial dance, a wondrous kin.
